In [1]:
from sqlalchemy import create_engine
import pandas as pd
from pandas.io import sql
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime 

In [2]:
def establish_conection(password, databasename = 'ConservatorDatabase', username = 'sa', hostname = 'localhost', port = '1433') :
    '''
    Establishes sqlalchemy connection to a mssql database

    Need to give it a password; default user = sa, port = 1433, hostname = localhost, databasename = ConservatorDatabase
    username, password, hostname, port, databasename

    returns engine
    '''
    engine_string = 'mssql+pymssql://' + username + ':' + password + '@' + hostname + ':' + port + '/' + databasename

    return create_engine(engine_string)

def read_sql_table(tablename):
    '''
    tablename as a string
    read table in pandas, return query as a dataframe
    '''

    engine = establish_conection(password='gr@vityI13')

    return sql.read_sql_table(tablename, engine)

In [3]:
df = read_sql_table('IncomeExpenseTransactions')
df2 = read_sql_table('TransactionCategories')

In [4]:
df.columns

Index([u'IncomeExpenseReportId', u'CaseFileReportId', u'AccountId',
       u'Description', u'Amount', u'TransactionDate', u'IsIncome',
       u'IsExpense', u'TransactionCategoryId', u'CreateDate', u'CreateUserId',
       u'LastModDate', u'LastModUserId', u'CourtOrderTransaction',
       u'TransactionPaymentTypeId', u'CheckNumber', u'SoldPropertyId',
       u'MortgagedRealEstateReportId', u'BoughtPropertyType', u'Amended',
       u'BoughtPropertyDescription', u'DateMovedFromCareFacility',
       u'RealEstateIncludesPropertyTaxes', u'RealEstatePropertyTaxesAmount',
       u'RealEstateIncludesPropertyInsurance',
       u'RealEstatePropertyInsuranceAmount',
       u'ProtectedPersonLivesAtCareFacility', u'TrustName'],
      dtype='object')

In [4]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

column_rename = {'IncomeExpenseReportId': 'IncExpId', 'CaseFileReportId': 'CasFil',
                 'TransactionDate': 'TranDt',
                 'TransactionCategoryId': 'TranCat', 'CourtOrderTransaction': 'CourtOrdTran',
                 'ProtectedPersonLivesAtCareFacility': 'CarFac'}

df.rename(columns=column_rename, inplace=True)

df['Expense'] = df['IsExpense'].astype(int)
df['CarFac'] = df['CarFac'].astype(int)
df['CourtOrdTran'] = df['CourtOrdTran'].astype(int)

del df['CreateUserId']
del df['LastModUserId']
del df['LastModDate']
del df['CheckNumber']
del df['Amended']
del df['TrustName']
del df['IsIncome']
del df['IsExpense']

In [5]:
d_tran = {}

trans_cat = df2['TransactionCategoryId'].unique()
for i in xrange(len(trans_cat)):
    d_tran[df2.iloc[i]['TransactionCategoryId']] = df2.iloc[i]['Description']

df['TranDescr'] = df['TranCat'].map(d_tran)


In [6]:
income_categories = df2[(df2['CategoryTypeId']==2) & (df2['TransactionCategoryId']!=245) & 
                        (df2['TransactionCategoryId']!=228) & 
                        (df2['TransactionCategoryId']!=249 )]['TransactionCategoryId'].tolist()

In [8]:
columns_to_keep = ['IncExpId', 'CasFil', 'Expense', 'Amount', 'TranDt', 'TranCat', 'TranDescr', 'CourtOrdTran', 'CarFac' ]
df = df[(df['TranCat'].isin(income_categories)) & (df['TranDt']> datetime.date(year=2010,month=1,day=1))
        & (df['TranDt']<datetime.date(2016, 1, 1))][columns_to_keep]

df['NumDays'] = (df['TranDt'] - df['TranDt'].min())/ np.timedelta64(1,'D')


df

,IncExpId,CasFil,Expense,Amount,TranDt,TranCat,TranDescr,CourtOrdTran,CarFac,NumDays
0,4673,4815,1,1250.00,2014-04-15,185,Care Facility,0,1,1562.0
1,4674,4815,1,104.00,2014-04-15,214,Personal Needs,0,0,1562.0
2,4675,4815,1,352.00,2014-04-15,240,Medical - Prescriptions,0,0,1562.0
3,4676,4816,1,500.00,2014-04-15,214,Personal Needs,0,0,1562.0
4,4677,4820,1,456.00,2014-04-15,239,Rent,0,0,1562.0
5,4678,4818,1,560.00,2014-04-16,239,Rent,0,0,1563.0
6,4679,4821,1,120.00,2014-04-16,239,Rent,0,0,1563.0
7,4680,4826,1,560.00,2014-04-16,239,Rent,0,0,1563.0
10,4683,4829,1,1200.00,2014-04-16,224,Taxes - Federal,0,0,1563.0
11,4684,4835,1,456.00,2014-04-16,239,Rent,0,0,1563.0


In [9]:
df.to_csv('expense.csv')

In [18]:
duration = df.groupby(by='CasFil')['NumDays'].max() - df.groupby(by='CasFil')['NumDays'].min()
NoCount = df.groupby(by='CasFil')['Amount'].count()
Total = df.groupby(by='CasFil')['Amount'].sum()

raw = {'Duration': list(duration), 'NumDataPts' : list(NoCount), 'Total' : Total}
explore = pd.DataFrame(raw, columns = ['Duration', 'NumDataPts',  'Total'], index=NoCount.index)
explore.reset_index(level=0, inplace=True)

In [19]:
df_piv = df.pivot_table(values=['Amount'], index=['CasFil'], columns='TranCat', aggfunc='count')
df_piv.fillna(0, inplace=True)
df_piv['CasFil'] = df_piv.index
df_piv

Amount                                                  ...          \
TranCat    179  180  181  182   183  184   185   186  187  188  ...     231   
CasFil                                                          ...           
4815       0.0  0.0  0.0  0.0   0.0  0.0   1.0   0.0  0.0  0.0  ...     0.0   
4816       0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4818       0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4820       0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4821       0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4826       0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4829       0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4832       2.0  0.0  1.0  0.0   2.0  2.0  13.0   1.0  1.0  1.0  ...     1.0   
4835       0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4841       0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4843       0.0  3.0  0.0  0.0   4.0  0.0  45.0   3.0  4.0  0.0  ...     0.0   
4844       0.0  0.0  0.0  0.0  12.0  0.0  13.0   0.0  2.0  0.0  ...     0.0   
4845       0.0  0.0  0.0  0.0   0.0  0.0   2.0   0.0  0.0  0.0  ...     0.0   
4846       0.0  0.0  0.0  0.0   2.0  1.0   5.0   0.0  0.0  0.0  ...     0.0   
4848       0.0  0.0  0.0  0.0   0.0  0.0  12.0   0.0  2.0  0.0  ...     0.0   
4851       0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4852       0.0  0.0  0.0  0.0   1.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4853       0.0  8.0  0.0  0.0   1.0  2.0  12.0   0.0  0.0  0.0  ...     0.0   
4854       0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4855       0.0  0.0  0.0  0.0   0.0  0.0  11.0   0.0  2.0  0.0  ...     0.0   
4859       0.0  0.0  0.0  0.0   1.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4860       0.0  0.0  0.0  0.0   0.0  1.0  13.0   0.0  0.0  0.0  ...     0.0   
4862       0.0  0.0  0.0  0.0   1.0  0.0  14.0   0.0  1.0  0.0  ...     0.0   
4864       2.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4865       0.0  0.0  1.0  0.0   0.0  0.0   0.0   0.0  8.0  0.0  ...     0.0   
4866       0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  3.0  0.0  ...     0.0   
4867       0.0  0.0  0.0  0.0   0.0  1.0   0.0   0.0  1.0  0.0  ...     0.0   
4868       0.0  0.0  0.0  0.0   1.0  1.0   0.0   0.0  0.0  0.0  ...    12.0   
4869       6.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
4871       2.0  0.0  0.0  0.0   2.0  0.0  12.0   0.0  2.0  0.0  ...     0.0   
...        ...  ...  ...  ...   ...  ...   ...   ...  ...  ...  ...     ...   
34317      0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
34323      0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
34334      0.0  0.0  0.0  0.0   0.0  1.0   0.0   0.0  0.0  0.0  ...     0.0   
34338      0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  1.0  ...     0.0   
34367      0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
34371      0.0  9.0  0.0  0.0   0.0  0.0   0.0  15.0  2.0  0.0  ...     0.0   
34373      0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  1.0  ...     0.0   
34374      0.0  0.0  0.0  0.0   3.0  1.0   0.0   0.0  2.0  1.0  ...     3.0   
34375      0.0  0.0  0.0  0.0   3.0  1.0   0.0   0.0  2.0  3.0  ...     0.0   
34377      0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
34406      0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
34421      0.0  0.0  0.0  0.0   1.0  0.0   0.0   0.0  0.0  0.0  ...     0.0   
34422      0.0  0.0  0.0  0.0   0.0  2.0   0.0   0.0  0.0  0.0  ...     0.0   
34423      0.0  0.0  0.0  0.0   0.0  0.0   0.0   0.0  2.0  0.0  ...     0.0   
34430      0.0  0.0  0.0  0.0   0.0  1.0   0.0   0.0  8.0  0.0  ...     0.0   
34432      0.0  0.0  0.0  0.0   0.0  0.0   2.0   0.0  0.0  0.0  ...     0.0   
34444      0.0  0.0  0.0  0.0   1.0  0.0   1.0   0.0  0.0 

In [20]:
df_piv = pd.merge(df_piv, explore, how='outer', left_on='CasFil', right_on='CasFil')
df_piv.fillna(0, inplace=True)
del df_piv[df_piv.columns.tolist()[-4]]
df_piv = df_piv.set_index('CasFil')

df_piv

,"(Amount, 179)","(Amount, 180)","(Amount, 181)","(Amount, 182)","(Amount, 183)","(Amount, 184)","(Amount, 185)","(Amount, 186)","(Amount, 187)","(Amount, 188)",...,"(Amount, 233)","(Amount, 234)","(Amount, 236)","(Amount, 238)","(Amount, 239)","(Amount, 240)","(Amount, 250)",Duration,NumDataPts,Total
CasFil,,,,,,,,,,,,,,,,,,,,,
4815,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3,1706.00
4816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,500.00
4818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,560.00
4820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,456.00
4821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,120.00
4826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,560.00
4829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1200.00
4832,2.0,0.0,1.0,0.0,2.0,2.0,13.0,1.0,1.0,1.0,...,25.0,1.0,1.0,1.0,24.0,6.0,0.0,454.0,177,96710.75
4835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,456.00


In [21]:
df_piv.to_csv('expense_count.csv')

In [22]:
df_piv = df.pivot_table(values=['Amount'], index=['CasFil'], columns='TranCat', aggfunc='sum')
df_piv.fillna(0, inplace=True)
df_piv['CasFil'] = df_piv.index
df_piv = pd.merge(df_piv, explore, how='outer', left_on='CasFil', right_on='CasFil')
df_piv.fillna(0, inplace=True)
del df_piv[df_piv.columns.tolist()[-4]]
df_piv = df_piv.set_index('CasFil')

df_piv

,"(Amount, 179)","(Amount, 180)","(Amount, 181)","(Amount, 182)","(Amount, 183)","(Amount, 184)","(Amount, 185)","(Amount, 186)","(Amount, 187)","(Amount, 188)",...,"(Amount, 233)","(Amount, 234)","(Amount, 236)","(Amount, 238)","(Amount, 239)","(Amount, 240)","(Amount, 250)",Duration,NumDataPts,Total
CasFil,,,,,,,,,,,,,,,,,,,,,
4815,0.00,0.00,0.00,0.0,0.00,0.00,1250.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.0,0.00,352.00,0.0,0.0,3,1706.00
4816,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,1,500.00
4818,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.0,560.00,0.00,0.0,0.0,1,560.00
4820,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.0,456.00,0.00,0.0,0.0,1,456.00
4821,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.0,120.00,0.00,0.0,0.0,1,120.00
4826,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.0,560.00,0.00,0.0,0.0,1,560.00
4829,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,1,1200.00
4832,202.04,0.00,181.00,0.0,197.45,2677.00,62504.51,186.00,187.00,188.00,...,771.18,234.00,236.0,238.0,2040.25,619.30,0.0,454.0,177,96710.75
4835,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.0,456.00,0.00,0.0,0.0,1,456.00


In [23]:
df_piv.to_csv('expense_total.csv')

In [24]:
df2[(df2['CategoryTypeId']==2) &(df2['TransactionCategoryId']!=245) & 
                        (df2['TransactionCategoryId']!=228) & 
                        (df2['TransactionCategoryId']!=249 )][['TransactionCategoryId', 'Description']].to_csv('expense_description.csv')

In [31]:
cols = df_piv.columns

In [54]:
df.pivot_table( index=['CasFil'], columns='TranCat', aggfunc='sum')

In [7]:
df2[df2['CategoryTypeId']==2][['TransactionCategoryId', 'Description']]

,TransactionCategoryId,Description
20,179,Asset Depreciation
21,180,Automobile - Gasoline
22,181,Automobile - Maintenance
23,182,Automobile - Payment
24,183,Bank Service Charges
25,184,Bond Premium
26,185,Care Facility
27,186,Charitable Donation
28,187,Clothing
29,188,Education
